# GPT-3.5-Turbo Model
Creating a question answering chatbot using GPT-3.5. Adapted from: https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_embeddings.ipynb

num_tokens() IS OFTEN USED IN KNOWLEDGE.PY (and other files??) WITHOUT SPECIFYING THE EMBEDDING MODEL.

In [1]:
# !pip install datasets, sentencepiece, transformers, accelerate, tiktoken, rouge_score, evaluate rouge_score bleu_score
import sys
sys.path.append("modules")
from modules.config import *
from modules.knowledge import *
from modules.chatbot import *
from modules.embedding_functions import *
from modules.data_extraction import *
from modules.data_preprocessing import *
from modules.gpt_ans_extraction import *
from modules.query import *
sys.path.remove("modules")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\point\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\point\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\point\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\point\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


No GPU available, using a CPU
No GPU available, using a CPU


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\point\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\point\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\point\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\point\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Create Knowledge Base

In [2]:
textbooks = ['Digital_Image_Processing_Textbook', 'Fundamentals_of_Digital_Image_Processing_Textbook']
CompVisionKnowledge = Knowledge(CHATBOT_TOPIC, gpt=True)
# for page in WIKI_PAGES:
#     CompVisionKnowledge.append_wikipedia_page(WIKI_PAGE)
for textbook in textbooks:
    CompVisionKnowledge.append_pdf(f'assets/knowledge/{textbook}.pdf', textbook)
CompVisionKnowledge.export_to_csv()
CompVisionKnowledge.df

Token indices sequence length is longer than the specified maximum sequence length for this model (515 > 512). Running this sequence through the model will result in indexing errors


The following PDF has been successfully added to the knowledge database: Digital_Image_Processing_Textbook (assets/knowledge/Digital_Image_Processing_Textbook.pdf)
The following PDF has been successfully added to the knowledge database: Fundamentals_of_Digital_Image_Processing_Textbook (assets/knowledge/Fundamentals_of_Digital_Image_Processing_Textbook.pdf)


,Source,Heading,Subheading,Page,Content,Section,Tokens,Embedding
0,Digital_Image_Processing_Textbook,,,0/1/2/3,\nGLOBAL \nEDITION\nDigital Image Processing\n...,Digital_Image_Processing_Textbook->Page(s)0/1/2/3,791.0,"[-0.021132638677954674, 0.025948533788323402, ..."
1,Digital_Image_Processing_Textbook,,,3/4/5/6/7/8/9,\nISBN 10: 1-292-22304-9\nISBN 13: 978-1-292-2...,Digital_Image_Processing_Textbook->Page(s)3/4/...,793.0,"[0.005928434897214174, 0.01934611052274704, -0..."
2,Digital_Image_Processing_Textbook,,,9/10/11,\nBackground 904\nPatterns and Pattern Classe...,Digital_Image_Processing_Textbook->Page(s)9/10/11,777.0,"[-0.011725608259439468, 0.021519940346479416, ..."
3,Digital_Image_Processing_Textbook,,,11/12,\nChapter 5: Revisions to this chapter were li...,Digital_Image_Processing_Textbook->Page(s)11/12,413.0,"[0.013898221775889397, 0.013536868616938591, -..."
4,Digital_Image_Processing_Textbook,,,12/13,\nChapter 12: This chapter underwent a major r...,Digital_Image_Processing_Textbook->Page(s)12/13,563.0,"[-0.005244759377092123, 0.018554115667939186, ..."
...,...,...,...,...,...,...,...,...
224,Fundamentals_of_Digital_Image_Processing_Textbook,,,341/342,"\nkey elements, 21\nmathematics of, 22 37\ncon...",Fundamentals_of_Digital_Image_Processing_Textb...,771.0,"[0.0032916401978582144, 0.0268571637570858, -0..."
225,Fundamentals_of_Digital_Image_Processing_Textbook,,,342/343/344,"\nMagnetic resonance imaging (MRI), 49\nMahala...",Fundamentals_of_Digital_Image_Processing_Textb...,640.0,"[-0.009114357642829418, 0.00879641529172659, 0..."
226,Fundamentals_of_Digital_Image_Processing_Textbook,,,344/345,"\nPeriodic square wave synthesis, 117\nPerspec...",Fundamentals_of_Digital_Image_Processing_Textb...,591.0,"[-0.011132902465760708, 0.001304946606978774, ..."
227,Fundamentals_of_Digital_Image_Processing_Textbook,,,345/346/347/348/349/350/351,"\nSalt and pepper noise, 46, 90, 91, 93\nSampl...",Fundamentals_of_Digital_Image_Processing_Textb...,780.0,"[0.015710284933447838, 0.001202668296173215, 0..."


### NQ Data Extraction and Preprocessing

In [ ]:
### Data extraction
training = AllData(cache_dir='/content/drive/MyDrive/Diss/Datasets', default='dev')
training.export_simplified_dataset(path="/content/drive/MyDrive/Diss/Output/simplified_dataset_validation_new.csv")

### Data preprocessing
model_name = "google/mt5-small" # This will dictate the model used for all the training
# model_name = "facebook/bart-large-xsum"
short_model_name = model_name.split('/')[1]
model_output_name = f"{short_model_name}_epochs_new_new"
HF_reference = f"psxjp5/{model_output_name}"
tokeniser = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, device_map="auto",
                                              torch_dtype="auto")
model.config.hidden_dropout_prob = 0.1 # For regularisation
tokeniser = add_special_tokens(tokeniser)
model.resize_token_embeddings(len(tokeniser))

training_data = TrainingData(tokeniser=tokeniser, save_dir=f'{OUTPUT_DIR}/all_data_{short_model_name}')

In [ ]:
# Load the dataset and extract only the answerable questions
all_data = load_from_disk(f'{OUTPUT_DIR}/all_data_{short_model_name}').shuffle(seed=9)
all_ans_data = all_data.filter(lambda row: (row["answer"] != NO_ANS_TOKEN))
all_ans_data

In [ ]:
# Iterate through the dataset and query GPT to extract a natural-language answer
splits = ['test', 'train']
for s in splits:
    all_timestamps = []
    responses = []
    # Obtain GPT answers/responses
    for i in tqdm(range(len(all_ans_data[s]))): # Due to the RPM
        all_timestamps = pause_if_needed(all_timestamps)

        formatted_text = format_request(all_ans_data[s][i])
        inputs = [
                {"role": "system", "content": f"You answer questions by only using a provided context."},
                {"role": "user", "content": formatted_text},
            ]
        response, all_timestamps = query_gpt(inputs, all_timestamps)
        responses.append(response.choices[0].message.content)

    # Export as HF dataset
    df_pandas = all_ans_data[s].to_pandas()
    df_pandas['gpt_ans'] = responses
    new_dataset = Dataset.from_pandas(df_pandas)
    new_dataset.save_to_disk(f"{OUTPUT_DIR}/{short_model_name}_{s}_split")

In [ ]:
# Now the new (GPT) answers can be merged with the original dataset
# Load the original data and GPT data generated above
train_dataset_pandas = all_data['train'].to_pandas()
test_dataset_pandas = all_data['test'].to_pandas()
# updated_train_dataset_pandas = load_from_disk(f"assets/{short_model_name}_train_split").to_pandas()
# updated_test_dataset_pandas = load_from_disk(f"assets/{short_model_name}_test_split").to_pandas()
updated_train_dataset_pandas = load_from_disk(f"{OUTPUT_DIR}/{short_model_name}_train_split").to_pandas()
updated_test_dataset_pandas = load_from_disk(f"{OUTPUT_DIR}/{short_model_name}_test_split").to_pandas()

# Merge on id and keep all rows
train_merged = pd.merge(train_dataset_pandas, updated_train_dataset_pandas[['id', 'gpt_ans']], on='id', how='outer')
test_merged = pd.merge(test_dataset_pandas, updated_test_dataset_pandas[['id', 'gpt_ans']], on='id', how='outer')

In [ ]:
# Remove any examples where it apparently had an answer but GPT couldn't extract oen
train_merged = train_merged[train_merged['gpt_ans'] != NO_ANS_TOKEN]
test_merged = test_merged[test_merged['gpt_ans'] !=  NO_ANS_TOKEN]

# Update the answers to match the GPT ones
train_merged.loc[train_merged['answer'] != NO_ANS_TOKEN, 'answer'] = train_merged.loc[train_merged['answer'] != NO_ANS_TOKEN, 'gpt_ans']
test_merged.loc[test_merged['answer'] != NO_ANS_TOKEN, 'answer'] = test_merged.loc[test_merged['answer'] != NO_ANS_TOKEN, 'gpt_ans']

merged_dataset = DatasetDict({
        "train": Dataset.from_pandas(train_merged),
        "test": Dataset.from_pandas(test_merged),
    })

# Maintain the answerable/non-answerable balance
def ensure_ans_non_ans_balance(dataset, dataset_splits=('training', 'validation'), proportion=0.3, seed=SEED):
    # Extracting the unanswerable examples
    no_ans = dataset.filter(lambda row: (row["answer"] == NO_ANS_TOKEN))
    good_ans = dataset.filter(lambda row: (row["answer"] != NO_ANS_TOKEN))

    # Discarding some unanswerable examples so the answer-no_ans ratio is favourable in each split
    processed_datasets_dict = {}
    for split in dataset_splits:
      num_no_ans = proportion*len(good_ans[split])/(1-proportion)
      no_ans_keep = no_ans[split].train_test_split(train_size=num_no_ans/len(no_ans[split]), seed=seed)['train']
      processed_datasets_dict[split] = concatenate_datasets([no_ans_keep, good_ans[split]])

    processed_dataset = DatasetDict({
            dataset_splits[0]: processed_datasets_dict[dataset_splits[0]],
            dataset_splits[1]: processed_datasets_dict[dataset_splits[1]],
        })
    shuffled_dataset = processed_dataset.shuffle(seed=seed)
    return shuffled_dataset

final_dataset = ensure_ans_non_ans_balance(merged_dataset, dataset_splits=['train', 'test'])

# As can be seen, the 80/20 train-test split has been maintained and there's no error responses
percentage = len(final_dataset['train'])/(len(final_dataset['train'])+len(final_dataset['test']))
print(f'Train/test split: {percentage}')
all_data.filter(lambda row: (row["answer"] == 'error'))

In [ ]:
# Finally, the questions/contexts and answers need to be tokenised to override previous values
def tokenise(data):
    # tokenize the inputs (questions and contexts)
    additional_cols = tokeniser(data['content'], data['question'], truncation=False)

    # tokenize the answers
    targets = tokeniser(text_target=data['answer'], truncation=False)

    #set labels
    additional_cols['labels'] = targets['input_ids']
    additional_cols['num_tokens'] = [len(row) for row in additional_cols["input_ids"]]
    return additional_cols

final_dataset = final_dataset.map(tokenise, batched = True)
final_dataset.save_to_disk(f'{OUTPUT_DIR}/all_data_{short_model_name}_gpt_updated')

# NQ Model Training

In [ ]:
# Load the GPT updated data
all_data = load_from_disk(f'{OUTPUT_DIR}/all_data_{short_model_name}_gpt_updated') # add or remove the suffix a required

# Training arguments/config
batch_size = 8
num_train_epochs = 20
logging_steps = len(all_data["train"]) // batch_size // 4 # Show the training loss with every epoch

args = Seq2SeqTrainingArguments(
    output_dir=model_output_name,
    logging_steps=logging_steps,
    evaluation_strategy="steps",
    save_steps=logging_steps,
    learning_rate=3e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    metric_for_best_model = 'bleu',
    load_best_model_at_end = True,
    push_to_hub=True,
    seed=9
)

# For model evaluation/metric computation
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_rouge(decoded_preds, decoded_labels):
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]

    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )

    # Extract the median scores
    results = {key: value * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in results.items()}

def compute_bleu(decoded_preds, decoded_labels, preds):
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = sacrebleu.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokeniser.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

def compute_meteor(decoded_preds, decoded_labels):
    result = meteor.compute(predictions=decoded_preds, references=decoded_labels)
    return {k: round(v, 4) for k, v in result.items()}

def compute_perc_noans_acc(decoded_preds, decoded_labels):
    total = len(decoded_labels)
    correct = sum(1 for pred, label in zip(decoded_preds, decoded_labels) if (pred == label and label==''))

    return {'No Ans Accuracy': round(correct/total, 4)*100}

def compute_cosine_similarity(decoded_preds, decoded_labels):
    cosine_sims = []
    for pred, label in zip(decoded_preds, decoded_labels):
      if label != '':  # i.e. Is answerable
        encoded_pred = GENERAL_EMBEDDING_MODEL.encode(pred)
        encoded_label = GENERAL_EMBEDDING_MODEL.encode(label)
        similarity = 1 - spatial.distance.cosine(encoded_pred, encoded_label)
        cosine_sims.append(similarity)

    mean = sum(cosine_sims)/len(cosine_sims)

    return {'Av Cosine Sim': round(mean, 4)}

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    # Decode generated answers
    decoded_preds = tokeniser.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokeniser.pad_token_id)
    # Decode target answers
    decoded_labels = tokeniser.batch_decode(labels, skip_special_tokens=True)

    # Compute metrics
    rouge_results = compute_rouge(decoded_preds, decoded_labels)
    bleu_results = compute_bleu(decoded_preds, decoded_labels, preds)
    meteor_results = compute_meteor(decoded_preds, decoded_labels)
    noans_acc = compute_perc_noans_acc(decoded_preds, decoded_labels)
    cosine_sim = compute_cosine_similarity(decoded_preds, decoded_labels)

    all_results = {**rouge_results, **bleu_results, **meteor_results, **noans_acc, **cosine_sim}

    return all_results

data_collator = DataCollatorForSeq2Seq(tokeniser, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=all_data["train"],
    eval_dataset=all_data["test"],
    data_collator=data_collator,
    tokenizer=tokeniser,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.1)]
)

trainer.train()
trainer.push_to_hub()
trainer.save_model(model_output_name)

### MLM Training

In [ ]:
# Extracting training content
for textbook in textbooks:
    all_text = ''.join(CompVisionKnowledge.df.loc[CompVisionKnowledge.df['Source']==textbook, 'Content'].tolist())
    with open(f'assets/knowledge/{textbook}.txt', "w") as f:
        f.write(all_text)

In [ ]:
# Read txt files
def read_txt(file_path):
    with open(file_path, "r") as file:
        text = file.read()
    return text

def get_text_dataset(path):
  dataset_obj = TextDataset(
        tokenizer = tokeniser,
        file_path = path,
        block_size = 512,
    )
  return dataset_obj

# Read documents from the directory
training_file = '/content/drive/MyDrive/Diss/Datasets/Digital_Image_Processing_Textbook.txt'
validation_file = '/content/drive/MyDrive/Diss/Datasets/Fundamentals_of_Digital_Image_Processing_Textbook.txt'
train_data = read_txt(training_file)
validation_data = read_txt(validation_file)
train_data = re.sub(r'\n+', '\n', train_data).strip()  # Remove excess newline characters
validation_data = re.sub(r'\n+', '\n', validation_data).strip()  # Remove excess newline characters

with open(f'{training_file}_formatted', "w") as f:
    f.write(train_data)
with open(f'{validation_file}_formatted', "w") as f:
    f.write(validation_data)

batch_size = 8
num_train_epochs = 50

# Train
gpt_tokeniser = GPT2Tokenizer.from_pretrained('gpt2')
gpt_model = GPT2LMHeadModel.from_pretrained('gpt2')
if gpt_tokeniser.pad_token is None:
    gpt_tokeniser.add_special_tokens({'pad_token': '[PAD]', 'mask_token': '[MASK]'})
    gpt_model.resize_token_embeddings(len(gpt_tokeniser))

data_collator = DataCollatorForLanguageModeling(
        tokenizer=gpt_tokeniser,
        mlm=True,
    )

train_dataset = get_text_dataset(f'{training_file}_formatted')
validation_dataset = get_text_dataset(f'{validation_file}_formatted')

HF_REFERENCE='mlm_new'
logging_steps = len(train_dataset) // batch_size // 4
training_args = TrainingArguments(
    output_dir=HF_REFERENCE,
    overwrite_output_dir=True,
    evaluation_strategy="steps",
    logging_steps=logging_steps,
    save_steps=logging_steps,
    per_device_train_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    metric_for_best_model = 'loss',
    load_best_model_at_end = True,
    save_total_limit=3,
    push_to_hub=True,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
)

trainer = Trainer(
    model=gpt_model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=0.05)]
)
trainer.train()
import math
# eval_results = trainer.evaluate()
# print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")
trainer.push_to_hub()
tokeniser.push_to_hub(f'psxjp5/{HF_REFERENCE}')
print('Finished training and pushed to hub!')

# Usage

In [3]:
# Todo:
# Check num_tokens usage and specifying the encoding model
CompVisionChatbot = ChatBot(CHATBOT_TOPIC, hf_reference=T5_QA_GPT_HF_REFERENCE, embedding='gpt')

In [4]:
# print(Query.ask_gpt('When did Universities begin teaching Computer Vision?', CompVisionChatbot, show_source=True))

print(Query.ask_finetuned('What is PCA?', CompVisionChatbot, show_source=True))  # What if the GPT knowledge sections are longer than 1024 tokens?? Need to account for this!

# 'What is PCA?'

C:\Users\point\OneDrive\Documents\Standard Files\University\Masters\Diss\MSc-Diss\code\modules\query.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.knowledge_used['Index'] = np.arange(len(self.knowledge_used)) + 1
C:\Users\point\OneDrive\Documents\Standard Files\University\Masters\Diss\MSc-Diss\code\modules\query.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.knowledge_used['Tokens'] = self.knowledge_used["Content"].apply(
C:\Users\point\OneDrive\Documents\Standard Files\University\Mas

The PCA is used to approximate the face.

To construct this answer, I used the following documents : 

1. Fundamentals_of_Digital_Image_Processing_Textbook->Page(s)278/279/280/281:
When we carry out a PCA on this ensemble of faces, we find that the eigenvalue spectrum
dies off qu...


C:\Users\point\OneDrive\Documents\Standard Files\University\Masters\Diss\MSc-Diss\code\modules\query.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.knowledge_used['Output'] = '\n\n' + self.knowledge_used['Index'].astype(str) + '. ' + self.knowledge_used[
